In [1]:
from utils.utils import *
from utils.utils_nn import *
from utils.utils_baseline_svm import *
from utils.Rnn_model import Rnn_model
# import kwargs dicts
from utils.kwargs_file import kwargs_neural_data_init, \
    kwargs_lin_data_init, kwargs_svm  # , kwargs_rnn

import numpy as np
np.random.seed(seed())

import tensorflow as tf
tf.reset_default_graph()

import os

from tensorflow.contrib import rnn

from sklearn.feature_extraction import DictVectorizer
from sklearn import svm

from collections import Counter, OrderedDict

import pandas as pd

import random
random.seed(seed())

import argparse

In [2]:
"""
Enable passing some keyword arguments from command line.
This does not affect the Jupyter notebook.
"""
# try:
parser = argparse.ArgumentParser()

parser.add_argument('--use_suggestions', action='store', dest='use_suggestions',
                    help='Should the algorithm use label suggestions '+\
                        '(False would mean using only the given labeled data)', 
                    type=str, 
                    default=None)

_StoreAction(option_strings=['--use_suggestions'], dest='use_suggestions', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Should the algorithm use label suggestions (False would mean using only the given labeled data)', metavar=None)

In [3]:
# results = parser.parse_args()
args, unknown = parser.parse_known_args()

if args.use_suggestions is not None:
    kwargs_neural_data_init.use_suggestions = str(args.use_suggestions)

In [4]:
"""

####### IF UNSURE, SKIP THIS CELL #######


Plug in manually the suggestion similarity threshold.
This is a very hacky solution, for getting all of the results for different thresholds.
Done this way because sending a for loop to the cluster takes too much overhead time,
on the otherhand, a for loop will use up too much RAM on local machine.
"""
# 0.7
# 0.8
# 0.9
# F

### DONE:
# 0.8
# F

kwargs_neural_data_init.use_suggestions = 'F'
# make sure SVM follows
kwargs_lin_data_init.use_suggestions = kwargs_neural_data_init.use_suggestions

In [5]:
# import the correct kwargs file, based on suggestion args
suggestion_convert_dict = \
    {'0.7': '07', 
     '0.8': '08', 
     '0.9': '09', 
     'F': '10'}

suggestion_str = \
    suggestion_convert_dict[kwargs_neural_data_init.use_suggestions]

import_kwargs_str = \
    'from utils.kwargs_file_{} import kwargs_rnn_GRU, kwargs_rnn_LSTM'.format(suggestion_str)

exec(import_kwargs_str)  # execute import statement

In [6]:
x_feed_train, y_feed_train, x_feed_val, y_feed_val,\
    char_int, char_int_inv, label_int, label_int_inv, \
    statistics_dict =\
    data_load_preprocess(**kwargs_neural_data_init)

Not using label suggestion data
The are 2028 observations
Sampling from allowed 47 labels
47 labels in the validation set, with
783 potential observation to draw from.
177 observations sampled for validation
606 observations for training
The ratio of validation to *training* is about 0.292


In [7]:
#### check if there are any strange splits between training and validation
#### e.g. labels (training and validation) don't match exactly
#### or not enough (or too many) observation were set aside for validation
y_train_counter = Counter(y_feed_train)
y_train_labs = list(y_train_counter.keys())
y_train_labs.sort()

y_val_counter = Counter(y_feed_val)
y_val_labs = list(y_val_counter.keys())
y_val_labs.sort()

assert y_train_labs==y_train_labs, 'Labels in training and validation set do not match!'

validation_training_ratios = []
for label in y_train_labs:
    val_num = y_val_counter[label]
    train_num = y_train_counter[label]
    ratio = float(val_num / train_num)
    validation_training_ratios.append(ratio)
#     print('{:.3f}'.format(ratio), label, val_num, train_num)

print('Minimal validation to trainig ration is {:.3f}'.format(min(validation_training_ratios)))
print('Maximal validation to trainig ration is {:.3f}'.format(max(validation_training_ratios)))

Minimal validation to trainig ration is 0.222
Maximal validation to trainig ration is 0.333


In [8]:
# returns np.arrays to feed into tf model
# training data
X_train, _, Y_train = index_transorm_xy(x=x_feed_train, 
                                        y=y_feed_train, 
                                        char_int=char_int, 
                                        label_int=label_int, 
                                        n_class=statistics_dict['n_class'])

# validation data
X_val, _, Y_val = index_transorm_xy(x=x_feed_val, 
                                    y=y_feed_val, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    n_class=statistics_dict['n_class'])

# write a metadata file for embeddings visualizer and create path string
embed_vis_path = write_embeddings_metadata(log_dir=kwargs_rnn_GRU.log_dir, 
                                           dictionary=char_int, 
                                           file_name='metadata.tsv')

In [9]:
kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

kwargs_rnn_GRU = nice_dict({**kwargs_rnn_GRU, **statistics_dict})
kwargs_rnn_LSTM = nice_dict({**kwargs_rnn_LSTM, **statistics_dict})

In [ ]:
"""
Train GRU and LSTM models
"""

In [ ]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

rnn_GRU_model.train()
rnn_GRU_model.close_session()

In [ ]:
##### LSTM model
hparam_str = make_hparam_string(**kwargs_rnn_LSTM)

rnn_LSTM_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_LSTM
                      )

rnn_LSTM_model.train()
rnn_LSTM_model.close_session()

In [ ]:
"""
Collect MRR per ATC code
"""

In [ ]:
collect_mrr_per_atc = []
collect_obs_num_per_atc = []

collected_results_per_atc_code_dir = 'rnn_results/results_per_atc_code/'

collected_obs_num_fname = collected_results_per_atc_code_dir + \
    'rnn_svm_obs_num_per_atc_code_sim_{}.p'.format(suggestion_str)
collected_mrr_fname = collected_results_per_atc_code_dir + \
    'rnn_svm_results_per_atc_code_sim_{}.p'.format(suggestion_str)

In [ ]:
"""
Initialize SVM and get MRR per ATC code
"""

In [10]:
v = DictVectorizer(sparse=True)

# load data
x_train_svm, x_val_svm, \
    y_train_svm, y_val_svm, \
    allowed_ngrams = \
    data_load_preprocess(**{**kwargs_lin_data_init, 
                            **{'linear_counters': False}})

# create and fit classifier
# create a sparse X matrix with character and n-grams features
X_train_svm = v.fit_transform([Counter(elem) for elem in x_train_svm])
X_val_svm = v.transform([Counter(elem) for elem in x_val_svm])

print('X_train_svm (sparse) matrix, of size {} by {} has been created.'
      .format(X_train_svm.get_shape()[0], 
              X_train_svm.get_shape()[1]))  # vectorized

svm_clf = svm.SVC(**kwargs_svm)

# print(svm_clf)

svm_clf.fit(X_train_svm, y_train_svm)

Not using label suggestion data
The are 2028 observations
Sampling from allowed 47 labels
47 labels in the validation set, with
783 potential observation to draw from.
177 observations sampled for validation
606 observations for training
The ratio of validation to *training* is about 0.292
X_train_svm (sparse) matrix, of size 606 by 4373 has been created.


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=2178, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
# array to hold log probabilities (takes a bit longer to calc)
pred_prob_svm = svm_clf.predict_log_proba(X_val_svm)
# makes y into array with the same shape as the log prob
y_val_svm_dense = y_to_dense(y=y_val_svm, 
                         classes=svm_clf.classes_)

recip_rank_array_svm = \
    mean_reciprocal_rank(
        y_val_svm_dense, 
        pred_prob_svm, 
        average=False)

In [ ]:
# get SVM MRR per ATC code

row_mrr = OrderedDict()
row_obs_num = OrderedDict()

# init a dict of empty lists
for atc in svm_clf.classes_:
    row_mrr[atc] = []
#     row_obs_num[atc] = []

# create lists of reciprocal ranks
for atc, recip_rank in zip(y_val_svm, recip_rank_array_svm):
    row_mrr[atc].append(recip_rank)

# calculate MRR
for k,v in row_mrr.items():
    row_mrr[k] = np.mean(v)
    row_obs_num[k] = len(v)

row_mrr['Similarity'] = int(suggestion_str) / 10
row_obs_num['Similarity'] = int(suggestion_str) / 10
row_mrr['Model type'] = 'SVM'
row_obs_num['Model type'] = 'SVM'

row_mrr.move_to_end('Model type', last=False)
row_obs_num.move_to_end('Model type', last=False)
row_mrr.move_to_end('Similarity', last=False)
row_obs_num.move_to_end('Similarity', last=False)

In [ ]:
# collect results
collect_mrr_per_atc.append(row_mrr)
collect_obs_num_per_atc.append(row_obs_num)

In [ ]:
"""
Initialize GRU, LSTM
"""

In [11]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

In [12]:
rnn_GRU_model.restore(kwargs_rnn_GRU.log_dir+hparam_str)

Loading variables from rnn_final_batch_trained_models/sim_10/GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=1.0,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-02,target_rep_weight=0.3/
Loading successful


In [13]:
##### LSTM model
hparam_str = make_hparam_string(**kwargs_rnn_LSTM)

rnn_LSTM_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_LSTM
                      )

In [14]:
rnn_LSTM_model.restore(kwargs_rnn_LSTM.log_dir+hparam_str)

Loading variables from rnn_final_batch_trained_models/sim_10/LSTM,bidir=F,noisy_tanh,learn_p=T,noise_alpha=1.15,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-03,target_rep_weight=0.5/
Loading successful


In [ ]:
"""
Log MRR per ATC code for RNNs
"""

In [ ]:
def get_mrr_per_atc(Rnn_model_inst):
    """
    Returns two OrderedDict's, MRR per ATC code and number of observations per ATC code
    Assumes that th efile was run with the same suggestion settings and x,y inputs
    """
    
    [accuracy, cost, recip_rank, top_k, logits] = \
        Rnn_model_inst.run_eval()

    # get an array of reciprocal ranks
    with tf.Session() as sess:
        recip_rank_array = \
            Rnn_model_inst.get_reciprocal_rank(
                logits=logits, 
                targets=kwargs_feed_dict_test['y']).eval()

    row_mrr = OrderedDict()
    
    row_obs_num = OrderedDict()
    row_obs_num['Similarity'] = int(suggestion_str) / 10
    row_obs_num['Model type'] = Rnn_model_inst.rnn_type
    
    # init a dict of empty lists
    for atc in set(y_feed_val):
        row_mrr[atc] = []

    # create lists of reciprocal ranks
    for atc, recip_rank in zip(y_feed_val, recip_rank_array):
        row_mrr[atc].append(recip_rank)

    # calculate MRR
    for k,v in row_mrr.items():
        row_mrr[k] = np.mean(v)
        row_obs_num[k] = len(v)

    row_mrr['Similarity'] = int(suggestion_str) / 10
    row_mrr['Model type'] = Rnn_model_inst.rnn_type

    row_mrr.move_to_end('Model type', last=False)
    row_mrr.move_to_end('Similarity', last=False)
    
    return row_mrr, row_obs_num

In [ ]:
# GRU: log MRR per ATC code
row_mrr_GRU, row_obs_num_GRU = \
    get_mrr_per_atc(rnn_GRU_model)

collect_mrr_per_atc.append(row_mrr_GRU)
collect_obs_num_per_atc.append(row_obs_num_GRU)

In [ ]:
# LSTM: log MRR per ATC code
row_mrr_LSTM, row_obs_num_LSTM = \
    get_mrr_per_atc(rnn_LSTM_model)

collect_mrr_per_atc.append(row_mrr_LSTM)
collect_obs_num_per_atc.append(row_obs_num_LSTM)

In [ ]:
# save the 3 results (SVM, LSTM, GRU) to pickle file format
# for each of the two statistics (MRR and count)
save(fname=collected_mrr_fname, 
     obj=collect_mrr_per_atc)

save(fname=collected_obs_num_fname, 
     obj=collect_obs_num_per_atc)

In [ ]:
"""
Construct data for varying prescription string length
Then log MRR
"""

In [15]:
mrr_per_seq_len_SVM = OrderedDict()
mrr_per_seq_len_SVM['Model'] = 'SVM'
mrr_per_seq_len_SVM['Similarity'] = int(suggestion_str) / 10

mrr_per_seq_len_GRU = OrderedDict()
mrr_per_seq_len_GRU['Model'] = 'GRU'
mrr_per_seq_len_GRU['Similarity'] = int(suggestion_str) / 10

mrr_per_seq_len_LSTM = OrderedDict()
mrr_per_seq_len_LSTM['Model'] = 'LSTM'
mrr_per_seq_len_LSTM['Similarity'] = int(suggestion_str) / 10

# make a copy of feed dicts to the RNN
kwargs_feed_dict_test_seq_len_rnn = kwargs_feed_dict_test

# makes y into array with the same shape as the log prob
y_val_svm_dense = y_to_dense(y=y_val_svm, 
                         classes=svm_clf.classes_)

# for seq_len in range(1,5+1):
for seq_len in range(1,kwargs_rnn_GRU.seq_len+1):
    print('Starting with sequence length {}'.format(seq_len))
    #### SVM data ####
    x_val_seq_len_svm = \
        keep_first_k_chars(input=x_val_svm, k=seq_len, 
                   model='linear', 
                   ngram_width=kwargs_lin_data_init.ngram_width, 
                   mk_ngrams=kwargs_lin_data_init.mk_ngrams, 
                   allowed_ngrams=allowed_ngrams)
    
    # create a sparse X matrix with character and n-grams features
    X_val_seq_len_svm = v.transform([Counter(elem) 
                                     for elem in x_val_seq_len_svm])
    
    # array to hold log probabilities (takes a bit longer to calc)
    pred_prob_seq_len_svm = \
        svm_clf.predict_log_proba(X_val_seq_len_svm)
    
    mrr_svm = \
        mean_reciprocal_rank(
            y_val_svm_dense, 
            pred_prob_seq_len_svm)
    
    ## log MRR for SVM ##
    mrr_per_seq_len_SVM[seq_len] = mrr_svm
    
    #### RNN data ####
    kwargs_feed_dict_test_seq_len_rnn['x'] = \
        keep_first_k_chars(input=X_val, k=seq_len, 
                               model='neural', 
                               char_int=char_int)
    
    # update test feed dictionaries
    rnn_GRU_model.update_test_dict(
        feed_dict_test=kwargs_feed_dict_test_seq_len_rnn)

    rnn_LSTM_model.update_test_dict(
        feed_dict_test=kwargs_feed_dict_test_seq_len_rnn)

    ## GRU MRR ##
    [_, _, mrr_GRU, _, _] = \
            rnn_GRU_model.run_eval()

    ## LSTM MRR ##
    [_, _, mrr_LSTM, _, _] = \
            rnn_LSTM_model.run_eval()
    
    ## log MRR for RNNs ##
    mrr_per_seq_len_GRU[seq_len] = mrr_GRU  # GRU
    mrr_per_seq_len_LSTM[seq_len] = mrr_LSTM  # LSTM

Starting with sequence length 1
Starting with sequence length 2
Starting with sequence length 3
Starting with sequence length 4
Starting with sequence length 5
Starting with sequence length 6
Starting with sequence length 7
Starting with sequence length 8
Starting with sequence length 9
Starting with sequence length 10
Starting with sequence length 11
Starting with sequence length 12
Starting with sequence length 13
Starting with sequence length 14
Starting with sequence length 15
Starting with sequence length 16
Starting with sequence length 17
Starting with sequence length 18
Starting with sequence length 19
Starting with sequence length 20
Starting with sequence length 21
Starting with sequence length 22
Starting with sequence length 23
Starting with sequence length 24
Starting with sequence length 25
Starting with sequence length 26
Starting with sequence length 27
Starting with sequence length 28
Starting with sequence length 29
Starting with sequence length 30
Starting with seque

In [16]:
mrr_per_seq_len_SVM

OrderedDict([('Model', 'SVM'),
             ('Similarity', 1.0),
             (1, 0.12897273312746454),
             (2, 0.13348470937339363),
             (3, 0.13905204961356313),
             (4, 0.14902429150980973),
             (5, 0.15500087165580131),
             (6, 0.17265490754140878),
             (7, 0.19342850818457338),
             (8, 0.21341119554515092),
             (9, 0.22152615497582026),
             (10, 0.2422486953461567),
             (11, 0.25970842139813521),
             (12, 0.2622546100585334),
             (13, 0.27735098100789629),
             (14, 0.29682048834791974),
             (15, 0.30138664227540757),
             (16, 0.31978994105400393),
             (17, 0.32711502151003441),
             (18, 0.33648139714425218),
             (19, 0.35623619396217499),
             (20, 0.35966155733044347),
             (21, 0.37935742106102505),
             (22, 0.39430264865966463),
             (23, 0.41985483949658275),
             (24, 0.451141

In [17]:
mrr_per_seq_len_GRU

OrderedDict([('Model', 'GRU'),
             ('Similarity', 1.0),
             (1, 0.45698613),
             (2, 0.65790641),
             (3, 0.83949959),
             (4, 0.92349344),
             (5, 0.9463343),
             (6, 0.95853901),
             (7, 0.96136391),
             (8, 0.96230561),
             (9, 0.96230561),
             (10, 0.96230561),
             (11, 0.96227062),
             (12, 0.96227062),
             (13, 0.96227062),
             (14, 0.96227062),
             (15, 0.96228719),
             (16, 0.96227062),
             (17, 0.96228719),
             (18, 0.96227062),
             (19, 0.96228719),
             (20, 0.96228719),
             (21, 0.96228719),
             (22, 0.96228719),
             (23, 0.96227062),
             (24, 0.96490711),
             (25, 0.96511203),
             (26, 0.96509546),
             (27, 0.97057337),
             (28, 0.97055686),
             (29, 0.97055686),
             (30, 0.97055686),
             (3

In [18]:
mrr_per_seq_len_LSTM

OrderedDict([('Model', 'LSTM'),
             ('Similarity', 1.0),
             (1, 0.12755091),
             (2, 0.15693289),
             (3, 0.18570159),
             (4, 0.21841292),
             (5, 0.23627855),
             (6, 0.25239888),
             (7, 0.25706753),
             (8, 0.26185051),
             (9, 0.26929688),
             (10, 0.30441821),
             (11, 0.31336775),
             (12, 0.30765662),
             (13, 0.30376804),
             (14, 0.30429891),
             (15, 0.30822772),
             (16, 0.32288352),
             (17, 0.31979552),
             (18, 0.3234525),
             (19, 0.33504808),
             (20, 0.34580347),
             (21, 0.35354766),
             (22, 0.37389383),
             (23, 0.38697669),
             (24, 0.39719197),
             (25, 0.41831061),
             (26, 0.43976787),
             (27, 0.43077242),
             (28, 0.4432151),
             (29, 0.45658615),
             (30, 0.44416267),
             (3

In [19]:
# save the 3 results (SVM, LSTM, GRU) to pickle file format
collected_results_per_seq_len_dir = 'rnn_results/results_per_seq_len/'
mrr_seq_len_fname = collected_results_per_seq_len_dir + \
    'rnn_svm_mrr_per_seq_len_sim_{}.p'.format(suggestion_str)

save(fname=mrr_seq_len_fname, 
     obj=[mrr_per_seq_len_SVM, 
          mrr_per_seq_len_LSTM, 
          mrr_per_seq_len_GRU])

In [ ]:
from tensorflow.contrib.learn.python.learn.utils.inspect_checkpoint import print_tensors_in_checkpoint_file

print_tensors_in_checkpoint_file(
    file_name='/home/yarden/git/Automated_categorization_medication/test_HERE' ,
                                 tensor_name=None,
#                                  all_tensors=True
                                                   )